In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Mounted at /content/drive/


In [2]:
import pandas as pd
import pandas as pd
balanced_sample = pd.read_csv('/content/drive/MyDrive/reddit/balanced_submission/balance_sample_sub.csv')

selected_columns = ['author', 'id',
       'selftext', 'subreddit', 'subreddit_id', 'title', 'Country',
       'selftext_NO_PUNC', 'title_NO_PUNC', 'fentanyl', 'count_fentanyl',
       'heroin', 'count_heroin', 'oxy', 'count_oxy', 'MDMA', 'count_MDMA',
       'INHALANTS', 'count_INHALANTS', 'GHB', 'count_GHB', 'COCAINE',
       'count_COCAINE', 'marijuana', 'count_marijuana', 'total_flags', 'match',
       'total_text_no_punc', 'total_text', 'concat', 'total_no_drug', 'year',
       'prediction_nb', 'xg_boost']

# Read the CSV file, selecting only the specified columns

no_included_samples = pd.read_csv('/content/drive/MyDrive/reddit/balanced_submission/no_include_sample_sub_nb_xg.csv', usecols=selected_columns)



<ipython-input-2-d98a2be3b459>:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  balanced_sample = pd.read_csv('/content/drive/MyDrive/reddit/balanced_submission/balance_sample_sub.csv')
<ipython-input-2-d98a2be3b459>:16: DtypeWarning: Columns (22) have mixed types. Specify dtype option on import or set low_memory=False.
  no_included_samples = pd.read_csv('/content/drive/MyDrive/reddit/balanced_submission/no_include_sample_sub_nb_xg.csv', usecols=selected_columns)


In [3]:
balanced_sample = balanced_sample[balanced_sample['match'] == 1]
balanced_sample

,Unnamed: 0.1,Unnamed: 0,author,id,selftext,subreddit,subreddit_id,title,Country,selftext_NO_PUNC,...,count_COCAINE,marijuana,count_marijuana,total_flags,match,total_text_no_punc,total_text,concat,total_no_drug,year
0,0,227,Greedy_fitbit,eiesev,"We have a small to medium sized garden, we hav...",CasualUK,t5_3j2jr,"Gardeners of the group, what should I be doing...",UK,We have a small to medium sized garden we hav...,...,0,trees,1,1,1,Gardeners of the group what should I be doing...,"Gardeners of the group, what should I be doing...",['trees'],Gardeners of the group what should I be doing ...,2020
1,1,236,dietageszeitung,eif9ns,,germantrees,t5_2tcsn,Klischeefreies Kiffen in Groningen: Das besond...,germany,,...,0,cannabis,1,1,1,Klischeefreies Kiffen in Groningen: Das besond...,Klischeefreies Kiffen in Groningen: Das besond...,['cannabis'],Klischeefreies Kiffen in Groningen: Das besond...,2020
2,2,353,crushingplanet,eigw1n,,paris,t5_2qkoi,Happy New Year Eve France 2020 | New year Eve ...,France,,...,0,NaN,0,1,1,Happy New Year Eve France 2020 | New year Eve ...,Happy New Year Eve France 2020 | New year Eve ...,['Eve'],Happy New Year France 2020 | New year Celebrat...,2020
3,3,402,sirgoofsalot,eihaf4,Hi guysTitle says it all really. My wife doesn...,AmsterdamEnts,t5_2vnvc,24 hours in Amsterdam with my conservative wif...,netherlands,Hi guysTitle says it all really My wife doesn...,...,0,weed,1,1,1,24 hours in Amsterdam with my conservative wif...,24 hours in Amsterdam with my conservative wif...,['weed'],24 hours in Amsterdam with my conservative wif...,2020
4,4,459,spuddy_spud_spud,eiho0b,,CasualUK,t5_3j2jr,Hope you appreciate my crude helicopter drawin...,UK,,...,0,NaN,0,1,1,Hope you appreciate my crude helicopter drawin...,Hope you appreciate my crude helicopter drawin...,['Eve'],Hope you appreciate my crude helicopter drawin...,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85883,85883,46284,jopea29,16whjcj,"Got some weed, has a weird high, dosent feel l...",uktrees,t5_2si9d,Sprayed?,uk,Got some weed has a weird high dosent feel l...,...,0,"cannabis, weed",2,2,1,Sprayed Got some weed has a weird high dosen...,"Sprayed? Got some weed, has a weird high, dose...","['cannabis', 'weed']",Sprayed Got some has a weird high dosent feel ...,2023
85884,85884,46340,calumb920505,16wj1o4,Yesterday it was very windy and my boxers got ...,CasualUK,t5_3j2jr,Neighbour took my boxers and hasn’t given them...,UK,Yesterday it was very windy and my boxers got ...,...,0,grass,1,1,1,Neighbour took my boxers and hasn’t given them...,Neighbour took my boxers and hasn’t given them...,['grass'],Neighbour took my boxers and hasn’t given them...,2023
85885,85885,46359,samreturned,16wjhrn,I recently joined this sub and commonly read o...,uktrees,t5_2si9d,Black Market THC Vape Oil,uk,I recently joined this sub and commonly read o...,...,0,spice,1,1,1,Black Market THC Vape Oil I recently joined th...,Black Market THC Vape Oil I recently joined th...,['spice'],Black Market THC Vape Oil I recently joined th...,2023
85886,85886,46416,FerretEuphoric9992,16wl58g,Ich habe kürzlich im Internet diese Website ei...,germantrees,t5_2tcsn,Weed aus dem Clear web?,germany,Ich habe kürzlich im Internet diese Website ei...,...,0,weed,1,1,1,Weed aus dem Clear web Ich habe kürzlich im In...,Weed aus dem Clear web? Ich habe kürzlich im I...,['weed'],aus dem Clear web Ich habe kürzlich im Interne...,2023


In [4]:
import pandas as pd

drug_summary = balanced_sample.columns.tolist()
count_columns = [col for col in drug_summary if col.startswith('count_')]

# Create the 'labels' column
balanced_sample['labels'] = balanced_sample.apply(
    lambda row: [col for col in count_columns if row[col] > 0],
    axis=1
)



In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
balanced_sample = balanced_sample[balanced_sample['total_no_drug'].notnull()]
# Sample data - replace this with your own dataset
texts = balanced_sample['total_no_drug'].tolist()
labels = balanced_sample['labels'].tolist()
#labels

mlb = MultiLabelBinarizer()
labels2 = mlb.fit_transform(labels)

In [6]:

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels2, test_size=0.2, random_state=42)

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)

# Train a Multinomial Naive Bayes classifier using MultiOutputClassifier for multilabel classification
nb_classifier = MultiOutputClassifier(MultinomialNB())
nb_classifier.fit(X_train_vec, y_train)

# Make predictions on the test set
y_pred = nb_classifier.predict(X_test_vec)

# Evaluate the performance
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

# Display classification report
print("Classification Report:\n", classification_report(y_test, y_pred, zero_division=1))


Accuracy: 0.83
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.01      0.02       605
           1       1.00      1.00      1.00         0
           2       1.00      0.00      0.00       109
           3       1.00      0.00      0.00       149
           4       1.00      0.05      0.10       188
           5       1.00      0.05      0.10       187
           6       0.85      1.00      0.92      6359
           7       1.00      0.02      0.05       421

   micro avg       0.85      0.79      0.82      8018
   macro avg       0.98      0.27      0.27      8018
weighted avg       0.88      0.79      0.74      8018
 samples avg       0.85      0.83      0.84      8018



In [7]:
class_nb = no_included_samples[no_included_samples['prediction_nb'] == 1]
class_nb_text = class_nb['total_text'].tolist()
no_sample_pre_vec = vectorizer.transform(class_nb_text)
no_sample_predictions = nb_classifier.predict(no_sample_pre_vec)
class_nb['prediction_nb_class'] = mlb.inverse_transform(no_sample_predictions)

no_included_samples = pd.merge(no_included_samples, class_nb[['id', 'prediction_nb_class']], how = 'left', on = 'id')


<ipython-input-7-d8eb4622a3aa>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  class_nb['prediction_nb_class'] = mlb.inverse_transform(no_sample_predictions)


In [54]:
!pip uninstall xgboost
!pip install xgboost

Found existing installation: xgboost 2.0.3
Uninstalling xgboost-2.0.3:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/xgboost-2.0.3.dist-info/*
    /usr/local/lib/python3.10/dist-packages/xgboost.libs/libgomp-a34b3233.so.1.0.0
    /usr/local/lib/python3.10/dist-packages/xgboost/*
Proceed (Y/n)? y
  Successfully uninstalled xgboost-2.0.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 3.5 MB/s eta 0:00:00


In [56]:
!pip install cupy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 12.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
Failed to build cupy
ERROR: Could not build wheels for cupy, which is required to install pyproject.toml-based projects


In [58]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1553  100  1553    0     0   6930      0 --:--:-- --:--:-- --:--:--  6933
sh: 9: nvidia-smi: not found
********************************************************************************
GPU is not enabled!
Open "Runtime" > "Change runtime type" and set "Hardware accelerator" to "GPU".
********************************************************************************


In [60]:
# Flatten the list of lists using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)
labels

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [10]:
X_test_vec

<7561x176550 sparse matrix of type '<class 'numpy.float64'>'
	with 349697 stored elements in Compressed Sparse Row format>

In [11]:

X_test_vec_np = X_test_vec.toarray()


In [9]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import recall_score
from sklearn.model_selection import GridSearchCV
import cupy as cp
from sklearn.metrics import make_scorer
from sklearn.metrics import precision_score

mlb = MultiLabelBinarizer()
# Sample data - replace this with your own dataset
texts = balanced_sample['total_no_drug'].tolist()
labels = balanced_sample['labels'].tolist()
labels = mlb.fit_transform(labels)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2, random_state=42)

# Convert text data to numerical vectors using TF-IDF
vectorizer = TfidfVectorizer(stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)
#X_test_vec_np = X_test_vec.toarray()
#X_test_vec_gpu = cp.asarray(X_test_vec_np)

# Create an XGBoost model with GPU support
params = {
    'objective': 'binary:logistic',
    'tree_method': 'hist',
     'device': 'cuda'  # Specify GPU as the device for training
    # Use GPU for tree construction
}

xgb_model = xgb.XGBClassifier(**params)


# Fit the model on the training data
xgb_model.fit(X_train_vec, y_train)

# Make predictions on the test data
y_pred = xgb_model.predict(X_test_vec)


y_pred_binary = (y_pred > 0.5).astype(int)



# Calculate recall with 'micro' average
precision_score_micro = precision_score(y_test, y_pred_binary, average='micro')
print(f'precision_score: {precision_score_micro}')

# Define parameter grid for grid search
#param_grid = {
#    'learning_rate': [0.1, 0.01],
#    'max_depth': [3, 5, 7],
#    'n_estimators': [100, 200]
#}

# Create a GridSearchCV object
# Create a custom scoring function for multilabel classification
#scorer = make_scorer(precision_score, average='micro')

# Create a GridSearchCV object with the custom scoring function
#grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, scoring=scorer, cv=3)


# Fit the grid search on the training data
#grid_search.fit(X_train_vec, y_train)

# Get the best parameters and the best model from the grid search
#best_params = grid_search.best_params_
#best_model = grid_search.best_estimator_




#y_pred_binary = (y_pred > 0.5).astype(int)

# Calculate recall with 'micro' average
#precision_score_micro = precision_score(y_test, y_pred_binary, average='micro')
#print(f'precision_score: {precision_score_micro}')


# Make predictions on the test data using the tuned model
#y_pred_tuned = best_model.predict(X_test_vec)
#y_pred_binary = (y_pred_tuned > 0.5).astype(int)

# Calculate recall for the tuned model
#precision_score_micro = precision_score(y_test, y_pred_binary, average='micro')
#print(f'precision_score: {precision_score_micro}')





/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [10:47:27] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


precision_score: 0.7616891203441639


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

KeyboardInterrupt: 

In [11]:
# Use CuPy for GPU-accelerated prediction
xg_no_pred = no_included_samples[no_included_samples['xg_boost'] == 1][['id', 'total_text']]

no_sample_pre = xg_no_pred['total_text'].tolist()
no_sample_pre_vec = vectorizer.transform(no_sample_pre)



In [12]:
check = cp.array(xgb_model.predict(no_sample_pre_vec))

predictions = check.get()

xg_no_pred['xg_boost_multi'] = mlb.inverse_transform(predictions)

In [14]:
no_included_samples2 = pd.merge(no_included_samples, xg_no_pred[['id', 'xg_boost_multi']], how = 'left', on = 'id')


,author,id,selftext,subreddit,subreddit_id,title,Country,selftext_NO_PUNC,title_NO_PUNC,fentanyl,...,match,total_text_no_punc,total_text,concat,total_no_drug,year,prediction_nb,xg_boost,prediction_nb_class,xg_boost_multi
0,WonderfulDog69,eiazwx,,portugal,t5_2qmie,"Primeiro vídeo do ano, bora. Um enorme 2020 a ...",Portugal,,Primeiro vídeo do ano bora Um enorme 2020 a ...,NaN,...,0,Primeiro vídeo do ano bora Um enorme 2020 a ...,"Primeiro vídeo do ano, bora. Um enorme 2020 a ...",[],Primeiro vídeo do ano bora Um enorme 2020 a to...,2020,0,0,NaN,NaN
1,meteoritee,eiazxp,Happy New Year 2020!Not only do we celebrate t...,CasualUK,t5_3j2jr,Happy New Year 2020!,UK,Happy New Year 2020 Not only do we celebrate t...,Happy New Year 2020,NaN,...,0,Happy New Year 2020 Happy New Year 2020 Not on...,Happy New Year 2020! Happy New Year 2020!Not o...,[],Happy New Year 2020 Happy New Year 2020 Not on...,2020,1,0,"(count_marijuana,)",NaN
2,Medical-Outcome,eib00z,,CasualUK,t5_3j2jr,Starting the next decade in a spoons 👍,UK,,Starting the next decade in a spoons 👍,NaN,...,0,Starting the next decade in a spoons 👍,Starting the next decade in a spoons 👍,[],Starting the next decade in a spoons 👍,2020,0,0,NaN,NaN
3,Rizlem,eib099,"I hope you have a happy new year, and your nex...",unitedkingdom,t5_2qhqb,HAPPY NEW YEAR!,UK,I hope you have a happy new year and your nex...,HAPPY NEW YEAR,NaN,...,0,HAPPY NEW YEAR I hope you have a happy new yea...,HAPPY NEW YEAR! I hope you have a happy new ye...,[],HAPPY NEW YEAR I hope you have a happy new yea...,2020,1,0,"(count_marijuana,)",NaN
4,Arculae,eib0d0,,uktrees,t5_2si9d,Happy New Year,uk,,Happy New Year,NaN,...,0,Happy New Year,Happy New Year,[],Happy New Year,2020,1,0,"(count_marijuana,)",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326814,Julial12,16wlxmr,,CasualUK,t5_3j2jr,Home alone here 😪,UK,,Home alone here 😪,NaN,...,0,Home alone here 😪,Home alone here 😪,[],Home alone here 😪,2023,1,0,"(count_marijuana,)",NaN
1326815,Allmychickenbois,16wlxow,"(SFW stuff only, you animals!)You know those n...",CasualUK,t5_3j2jr,What do you do to help yourself fall asleep?,UK,SFW stuff only you animals You know those ni...,What do you do to help yourself fall asleep,NaN,...,0,What do you do to help yourself fall asleep SF...,What do you do to help yourself fall asleep? (...,[],What do you do to help yourself fall asleep SF...,2023,1,0,"(count_marijuana,)",NaN
1326816,Free_Price3574,16wlyso,"Hej!Jag har en situation som precis hände, och...",sweden,t5_2qofe,Granne (Ordförande för BRFen) gick in i min lä...,Sweden,Hej Jag har en situation som precis hände och...,Granne Ordförande för BRFen gick in i min lä...,NaN,...,0,Granne Ordförande för BRFen gick in i min lä...,Granne (Ordförande för BRFen) gick in i min lä...,[],Granne Ordförande för BRFen gick in i min läge...,2023,0,0,NaN,NaN
1326817,Remarkable_Put_7952,16wm2d8,,thenetherlands,t5_30hrx,Den Haag (1929),netherlands,,Den Haag 1929,NaN,...,0,Den Haag 1929,Den Haag (1929),[],Den Haag 1929,2023,0,0,NaN,NaN


In [15]:
no_included_samples2[no_included_samples2['xg_boost'] == 1]

,author,id,selftext,subreddit,subreddit_id,title,Country,selftext_NO_PUNC,title_NO_PUNC,fentanyl,...,match,total_text_no_punc,total_text,concat,total_no_drug,year,prediction_nb,xg_boost,prediction_nb_class,xg_boost_multi
13,DaSaYaMa,eib3vb,"Bei uns in der Straße, Anliegerstraße und sehr...",de,t5_22i0,Ich hatte meinen deutschesten Moment des Jahre...,Germany,Bei uns in der Straße Anliegerstraße und sehr...,Ich hatte meinen deutschesten Moment des Jahre...,NaN,...,0,Ich hatte meinen deutschesten Moment des Jahre...,Ich hatte meinen deutschesten Moment des Jahre...,[],Ich hatte meinen deutschesten Moment des Jahre...,2020,1,1,"(count_marijuana,)","(count_marijuana,)"
16,Space-manatee,eib5un,,CasualUK,t5_3j2jr,People trying to see the London New Years’ fir...,UK,,People trying to see the London New Years’ fir...,NaN,...,0,People trying to see the London New Years’ fir...,People trying to see the London New Years’ fir...,[],People trying to see the London New Years’ fir...,2020,1,1,"(count_marijuana,)","(count_MDMA, count_marijuana)"
20,jamesg007,eib6vm,Hi there—I just moved to Luxembourg today (31 ...,Luxembourg,t5_2ryqf,"Local listings, like Yelp for Luxembourg?",Luxembourg,Hi there—I just moved to Luxembourg today 31 ...,Local listings like Yelp for Luxembourg,NaN,...,0,Local listings like Yelp for Luxembourg Hi th...,"Local listings, like Yelp for Luxembourg? Hi t...",[],Local listings like Yelp for Luxembourg Hi the...,2020,1,1,"(count_marijuana,)","(count_marijuana,)"
28,laurcifers,eibbkw,,CasualUK,t5_3j2jr,"starting the month, year and decade with a pin...",UK,,starting the month year and decade with a pin...,NaN,...,0,starting the month year and decade with a pin...,"starting the month, year and decade with a pin...",[],starting the month year and decade with a pint...,2020,1,1,"(count_marijuana,)","(count_MDMA, count_marijuana)"
33,albertoorea,eibdcf,,AmsterdamEnts,t5_2vnvc,Brought this back to LA. Thanks for the best 1...,netherlands,,Brought this back to LA Thanks for the best 1...,NaN,...,0,Brought this back to LA Thanks for the best 1...,Brought this back to LA. Thanks for the best 1...,[],Brought this back to LA Thanks for the best 10...,2020,1,1,"(count_marijuana,)","(count_marijuana,)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1326796,FerretEuphoric9992,16wl58g,Ich habe kürzlich im Internet diese Website ei...,germantrees,t5_2tcsn,Weed aus dem Clear web?,germany,Ich habe kürzlich im Internet diese Website ei...,Weed aus dem Clear web,NaN,...,1,Weed aus dem Clear web Ich habe kürzlich im In...,Weed aus dem Clear web? Ich habe kürzlich im I...,['weed'],aus dem Clear web Ich habe kürzlich im Interne...,2023,1,1,"(count_marijuana,)","(count_marijuana,)"
1326797,cyberoscar,16wl6aq,Jag inser att detta bara är en i mängden posts...,sweden,t5_2qofe,Min Take,Sweden,Jag inser att detta bara är en i mängden posts...,Min Take,NaN,...,1,Min Take Jag inser att detta bara är en i mäng...,Min Take Jag inser att detta bara är en i mäng...,['cannabis'],Min Take Jag inser att detta bara är en i mäng...,2023,0,1,NaN,"(count_fentanyl, count_heroin, count_marijuana..."
1326804,Zattlaegott,16wlg27,hej jag och en polare är extremt sugna på att ...,droger,t5_r5ugy,Hej någon som har har testat dmt och kan berät...,danish,hej jag och en polare är extremt sugna på att ...,Hej någon som har har testat dmt och kan berät...,NaN,...,0,Hej någon som har har testat dmt och kan berät...,Hej någon som har har testat dmt och kan berät...,[],Hej någon som har har testat dmt och kan berät...,2023,1,1,"(count_marijuana,)","(count_fentanyl, count_heroin, count_marijuana)"
1326810,Snopptorp,16wlt3z,mina föräldrar Cathca mig på att dricka men ja...,droger,t5_r5ugy,syns 6 bloss på piss,danish,mina föräldrar Cathca mig på att dricka men ja...,syns 6 bloss på piss,NaN,...,0,syns 6 bloss på piss mina föräldrar Cathca mig...,syns 6 bloss på piss mina föräldrar Cathca mig...,[],syns 6 bloss på piss mina föräldrar Cath

In [18]:
no_included_samples2.to_csv('/content/drive/MyDrive/reddit/balanced_comment/no_inc_samp_sub_nb_xg_drug_type.csv')